## This notebook contains Python code for analyzing data relevant to airlines

In [2]:
import pandas as pd
import numpy as np

import pymysql as mysql
import getpass

import matplotlib.pyplot as plt
import seaborn as sns

We need to connect to MySQL server, and specifically connect to the `ads507airlines` database. Then we verify the connection with a SQL command to show all tables.

In [3]:
conn=mysql.connect(host='localhost',
                   port=int(3306),
                   user='root',
                   passwd=getpass.getpass('Enter password: '),
                   db='ads507airlines')

Enter password:  ···············


In [5]:
tableNames = pd.read_sql("""SHOW TABLES""", conn)

tableNames

,Tables_in_ads507airlines
0,airlines_all2018
1,airlines_all2019
2,airlines_all2020
3,airlines_all2021
4,airlines_all2022
...,...
144,yearly_timedelay_2018
145,yearly_timedelay_2019
146,yearly_timedelay_2020
147,yearly_timedelay_2021


In [6]:
for i in tableNames['Tables_in_ads507airlines']:
    print(i)

airlines_all2018
airlines_all2019
airlines_all2020
airlines_all2021
airlines_all2022
daily_delays_iah2018
daily_delays_iah2019
daily_delays_iah2020
daily_delays_iah2021
daily_delays_iah2022
daily_delays_jfk2018
daily_delays_jfk2019
daily_delays_jfk2020
daily_delays_jfk2021
daily_delays_jfk2022
daily_delays_san2018
daily_delays_san2019
daily_delays_san2020
daily_delays_san2021
daily_delays_san2022
daily_delays_sea2018
daily_delays_sea2019
daily_delays_sea2020
daily_delays_sea2021
daily_delays_sea2022
daily_weather_iah2018
daily_weather_iah2019
daily_weather_iah2020
daily_weather_iah2021
daily_weather_iah2022
daily_weather_jfk2018
daily_weather_jfk2019
daily_weather_jfk2020
daily_weather_jfk2021
daily_weather_jfk2022
daily_weather_san2018
daily_weather_san2019
daily_weather_san2020
daily_weather_san2021
daily_weather_san2022
daily_weather_sea2018
daily_weather_sea2019
daily_weather_sea2020
daily_weather_sea2021
daily_weather_sea2022
delays_weather_iah2018
delays_weather_iah2019
delays_we

In [18]:
query_string = """
SELECT * FROM yearly_Timedelay_2018
UNION
SELECT * FROM yearly_Timedelay_2019
UNION
SELECT * FROM yearly_Timedelay_2020
UNION
SELECT * FROM yearly_Timedelay_2021
UNION
SELECT * FROM yearly_Timedelay_2022;"""

timedelay_df = pd.read_sql(query_string, conn)

timedelay_df

,carrier,origin,dest,year,AVGdepdelay,AVGarrdelay,Flight_Time
0,HA,JFK,HNL,2018,1348.0,1316.0,Early Morning
1,OO,IAH,CHS,2018,1295.0,1336.0,Morning
2,NK,IAH,FLL,2018,1283.0,1301.0,Morning
3,NK,SAN,ORD,2018,1200.5,1217.0,Morning
4,NK,SAN,DFW,2018,1166.0,1177.5,Afternoon
...,...,...,...,...,...,...,...
16280,G4,SAN,AZA,2022,-10.5,6.0,Early Morning
16281,YV,IAH,CRP,2022,-11.0,-6.0,Morning
16282,UA,IAH,BDL,2022,-11.5,-18.0,Evening
16283,UA,IAH,PIT,2022,-12.0,-10.0,Evening


### Example for airlines wanting to understand typical flight delays to specific destinations

In [29]:
average_delays_df = timedelay_df.groupby(['carrier','dest'])[['AVGdepdelay','AVGarrdelay']].mean().reset_index()

np.round(average_delays_df.rename(columns={'AVGdepdelay':"Avg Departure delay",
                                           'AVGarrdelay':'Avg Arrival delay'}),1)

,carrier,dest,Avg Departure delay,Avg Arrival delay
0,9E,ACK,57.3,55.3
1,9E,ATL,36.6,29.9
2,9E,BGR,82.6,72.3
3,9E,BNA,18.2,5.5
4,9E,BOS,6.0,-3.9
...,...,...,...,...
905,YX,SYR,20.2,4.0
906,YX,TPA,-5.4,-13.6
907,YX,TUL,4.9,0.8
908,YX,TYS,13.3,11.0


### Example for airlines wanting to understand typical flight delays out of specific airports

In [30]:
average_delays_df = timedelay_df.groupby(['carrier','origin'])[['AVGdepdelay','AVGarrdelay']].mean().reset_index()

np.round(average_delays_df.rename(columns={'AVGdepdelay':"Avg Departure delay",
                                           'AVGarrdelay':'Avg Arrival delay'}),1)

,carrier,origin,Avg Departure delay,Avg Arrival delay
0,9E,IAH,60.9,51.9
1,9E,JFK,38.6,28.3
2,AA,IAH,60.4,55.2
3,AA,JFK,38.3,29.1
4,AA,SAN,64.0,56.4
5,AA,SEA,87.3,78.6
6,AS,IAH,58.8,58.0
7,AS,JFK,28.5,19.2
8,AS,SAN,47.1,41.9
9,AS,SEA,20.6,15.6


### Example reviewing delays to destinations from specific airports, regardless of airline

In [34]:
average_delays_df = timedelay_df.groupby(['dest','origin'])[['AVGdepdelay','AVGarrdelay']].mean().reset_index()

np.round(average_delays_df.rename(columns={'AVGdepdelay':"Avg Departure delay",
                                           'AVGarrdelay':'Avg Arrival delay'}),1)

,dest,origin,Avg Departure delay,Avg Arrival delay
0,ABI,IAH,181.6,148.4
1,ABQ,IAH,43.1,39.3
2,ABQ,JFK,10.0,-6.1
3,ABQ,SAN,11.9,7.0
4,ABQ,SEA,9.6,3.5
...,...,...,...,...
387,TYS,IAH,31.9,25.9
388,VCT,IAH,127.4,97.3
389,VPS,IAH,90.0,86.7
390,XNA,IAH,43.2,38.0
